Common imports

In [114]:
%env PGE_DB_NAME = pgequity
%env PGE_DB_USER = pgequity
%env PGE_DB_PASSWORD = pgequity
%env PGE_DB_HOST = localhost
%env PGE_DB_PORT = 5432

env: PGE_DB_NAME=pgequity
env: PGE_DB_USER=pgequity
env: PGE_DB_PASSWORD=pgequity
env: PGE_DB_HOST=localhost
env: PGE_DB_PORT=5432


In [115]:
import polars as pl
import pandas as pd
from psycopg_pool import ConnectionPool
import os
import time
import numpy as np
import scipy as sp




_pgeparams = {
    'dbname': os.environ.get("PGE_DB_NAME"),
    'user': os.environ.get("PGE_DB_USER"),
    'password': os.environ.get("PGE_DB_PASSWORD"),
    'host': os.environ.get("PGE_DB_HOST"),
    'port': os.environ.get("PGE_DB_PORT"),
}

PGEPOOL = ConnectionPool(conninfo=f"postgresql://{_pgeparams['user']}:{_pgeparams['password']}@{_pgeparams['host']}:{_pgeparams['port']}/{_pgeparams['dbname']}", max_size=5)


Read data

In [121]:
df = None

#warmup
with PGEPOOL.connection() as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT index_date, close FROM data_points WHERE symbol = 'IDXGSPC' ORDER BY index_date ASC")
        data = cursor.fetchall()
        df = pl.DataFrame(data, schema=[("index_date", pl.Date), ("close", pl.Float64)], orient="row")

dfval = df.select(
    pl.col("index_date"),
    pl.col("close"),
    pl.col("close").diff(-1).lt(0).alias("up_next"),
    pl.col("close").diff(1).gt(0).alias("up_prev"),
)


print(df.tail(10))
print(dfval.tail(10))

dfvalser = df.to_series(1)
dfisup = dfvalser.diff(-1).lt(0)
dfgainonly = dfvalser * dfisup

dfnew = pl.DataFrame({
    "index_date": df.to_series(0),
    "close": dfvalser,
    "up_next": dfisup,
    "gain_only": dfgainonly
})

print(dfnew.tail(10))

dfcorr = dfnew.select(
    pl.col("index_date"),
    pl.col("close"),
    pl.col("up_next"),
    pl.rolling_corr(a = pl.col("close"), b = pl.col("up_next"), window_size=10).alias("corr"),
    pl.rolling_corr(a = pl.col("up_next"), b = pl.col("gain_only"), window_size=20).alias("corr20"),
)


print(dfcorr.tail(10))



shape: (10, 2)
┌────────────┬─────────────┐
│ index_date ┆ close       │
│ ---        ┆ ---         │
│ date       ┆ f64         │
╞════════════╪═════════════╡
│ 2024-10-28 ┆ 5823.52002  │
│ 2024-10-29 ┆ 5832.919922 │
│ 2024-10-30 ┆ 5813.669922 │
│ 2024-10-31 ┆ 5705.450195 │
│ 2024-11-01 ┆ 5728.799805 │
│ 2024-11-04 ┆ 5712.689941 │
│ 2024-11-05 ┆ 5782.759766 │
│ 2024-11-06 ┆ 5929.040039 │
│ 2024-11-07 ┆ 5973.100098 │
│ 2024-11-08 ┆ 5995.540039 │
└────────────┴─────────────┘
shape: (10, 4)
┌────────────┬─────────────┬─────────┬─────────┐
│ index_date ┆ close       ┆ up_next ┆ up_prev │
│ ---        ┆ ---         ┆ ---     ┆ ---     │
│ date       ┆ f64         ┆ bool    ┆ bool    │
╞════════════╪═════════════╪═════════╪═════════╡
│ 2024-10-28 ┆ 5823.52002  ┆ true    ┆ true    │
│ 2024-10-29 ┆ 5832.919922 ┆ false   ┆ true    │
│ 2024-10-30 ┆ 5813.669922 ┆ false   ┆ false   │
│ 2024-10-31 ┆ 5705.450195 ┆ true    ┆ false   │
│ 2024-11-01 ┆ 5728.799805 ┆ false   ┆ true    │
│ 2024-11-04 ┆ 5